# VGG16

In [1]:
# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam

# Veri yolları
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Çift_Grafikler/500_TRAİN_DATASET_ÇİFTGRAFİK_512x512/Sağlıklı_Çift_Grafik_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Çift_Grafikler/500_TRAİN_DATASET_ÇİFTGRAFİK_512x512/Okralı_Çift_Grafik_512x512"


# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Zararlı sınıf için verileri yükleme
for dosya_adi in os.listdir(okra_klasoru):
	dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
	img = load_img(dosya_yolu, target_size=(256, 256))
	img = img_to_array(img)
	#img = preprocess_vgg16(img)  # VGG16 için ön işleme
	X.append(img)
	y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme
for dosya_adi in os.listdir(normal_klasoru):
	dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
	img = load_img(dosya_yolu, target_size=(256, 256))
	img = img_to_array(img)
	#img = preprocess_vgg16(img)  # VGG16 için ön işleme
	X.append(img)
	y.append(1)  # Normal sınıf

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = False

# Tam bağlı katmanları oluşturma
model = Sequential([
	vgg16_model,
	Flatten(),
	Dense(128, activation='relu'),
	Dense(1, activation='sigmoid')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.001)  # Öğrenme oranını düşürme
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.1)

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')



58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/40
23/23 [==============================] - 19s 375ms/step - loss: 7.0984 - accuracy: 0.5833 - val_loss: 0.9267 - val_accuracy: 0.6500
Epoch 2/40
23/23 [==============================] - 3s 151ms/step - loss: 0.7451 - accuracy: 0.7181 - val_loss: 0.7964 - val_accuracy: 0.7375
Epoch 3/40
23/23 [==============================] - 4s 162ms/step - loss: 0.3900 - accuracy: 0.8236 - val_loss: 0.5321 - val_accuracy: 0.7375
Epoch 4/40
23/23 [==============================] - 4s 162ms/step - loss: 0.2859 - accuracy: 0.8819 - val_loss: 0.8079 - val_accuracy: 0.6250
Epoch 5/40
23/23 [==============================] - 4s 166ms/step - loss: 0.2604 - accuracy: 0.8944 - val_loss: 0.4538 - val_accuracy: 0.8250
Epoch 6/40
23/23 [==============================] - 4s 163ms/step - loss: 0.2698 - accuracy: 0.8875 - val_loss: 0.4911 - val_accuracy: 0.7750
Epoch 7/40
23/23 [==============================] - 4s 163ms/step - loss: 0.2221 -

In [4]:
from tensorflow.keras.models import load_model
model.save("/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/saved_model/Deneme_Çift_VGG16_Model_1_.keras")

# Daha Karmaşık Bir VGG16 With Blurriness and Dropout + Early Stopping

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage import gaussian_filter
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2

# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')

# Veri yolları
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Sağlıklı_Çift_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Okralı_Çift_512x512"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Fonksiyon: Görüntü yükleme ve bulanıklık ekleme
def load_and_blur_image(image_path, target_size=(256, 256)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    blurred_img_array = gaussian_filter(img_array, sigma=1.5)  # Bulanıklık ekleme
    return blurred_img_array

# Zararlı sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(okra_klasoru):
    dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(normal_klasoru):
    dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(1)  # Normal sınıf


# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = True

# Yeni katmanları oluşturun
model = Sequential([
    vgg16_model,
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout katmanı ekleyerek aşırı uyumun önüne geçebilirsiniz
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Modeli derleme
optimizer = Adam(learning_rate=0.0001)  # Öğrenme oranını düşürme
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping callback'i oluşturun
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme
history = model.fit(X_train, y_train, batch_size=32, epochs=40, validation_split=0.2, callbacks=[early_stopping])

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch 1/40
16/16 [==============================] - 66s 2s/step - loss: 0.7696 - accuracy: 0.5294 - val_loss: 0.9826 - val_accuracy: 0.4766
Epoch 2/40
16/16 [==============================] - 9s 554ms/step - loss: 0.7068 - accuracy: 0.5216 - val_loss: 0.6906 - val_accuracy: 0.5234
Epoch 3/40
16/16 [==============================] - 9s 553ms/step - loss: 0.6934 - accuracy: 0.5216 - val_loss: 0.6934 - val_accuracy: 0.4766
Epoch 4/40
16/16 [==============================] - 9s 538ms/step - loss: 0.6930 - accuracy: 0.5235 - val_loss: 0.6950 - val_accuracy: 0.4766
Epoch 5/40
16/16 [==============================] - 9s 535ms/step - loss: 0.6945 - accuracy: 0.5020 - val_loss: 0.6935 - val_accuracy: 0.4766
Epoch 6/40
16/16 [==============================] - 8s 527ms/step - loss: 0.6927 - accuracy: 0.5059 - val_loss: 0.6939 - val_accuracy: 0.4766
Epoch 7/40
5/5 [===

# Model Save

In [ ]:
from tensorflow.keras.models import load_model
model.save("/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/saved_model/400TrainDataDeneme_Daha_Karmaşık_VGG16_Blur_Dropout_Early_Model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# With Blurriness and Data Augmentation + Early Stopping




In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage import gaussian_filter
from keras.callbacks import EarlyStopping
import cv2
from keras.layers import Dropout

# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')

# Veri yolları
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Sağlıklı_Çift_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Okralı_Çift_512x512"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Fonksiyon: Görüntü yükleme ve bulanıklık ekleme
def load_and_blur_image(image_path, target_size=(256, 256)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    blurred_img_array = gaussian_filter(img_array, sigma=1.5)  # Bulanıklık ekleme
    return blurred_img_array

# Zararlı sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(okra_klasoru):
    dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(normal_klasoru):
    dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(1)  # Normal sınıf


# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#def add_blur(image):
 #   blurred_image = gaussian_filter(image, sigma=1.5)  #BU UZATIR İŞLEM SÜRESİNİ
  #  return blurred_image

# Veri artırımı için ImageDataGenerator oluşturma
datagen = ImageDataGenerator(
    shear_range=0.1,
    preprocessing_function=lambda img: cv2.GaussianBlur(img, (5, 5), sigmaX=1.5),  # Gaussian blur eklemek
    #randomcrop dene
)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = True

# Tam bağlı katmanları oluşturma
model = Sequential([
	vgg16_model,
	Flatten(),
	Dense(128, activation='relu'),
  Dropout(rate=0.5),
	Dense(1, activation='sigmoid')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.001)  # Öğrenme oranını düşürme
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping callback'i oluşturun
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme
history = model.fit(datagen.flow(X_train, y_train, batch_size=16), epochs=40, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')

Mounted at /content/gdrive


KeyboardInterrupt: 

# With Blurriness and Dropout + Early Stopping



In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage import gaussian_filter
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2

# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')

# Veri yolları
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Sağlıklı_Çift_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Okralı_Çift_512x512"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Fonksiyon: Görüntü yükleme ve bulanıklık ekleme
def load_and_blur_image(image_path, target_size=(256, 256)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    blurred_img_array = gaussian_filter(img_array, sigma=1.5)  # Bulanıklık ekleme
    return blurred_img_array

# Zararlı sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(okra_klasoru):
    dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(normal_klasoru):
    dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(1)  # Normal sınıf


# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = True

# Tam bağlı katmanları oluşturma
model = Sequential([
	vgg16_model,
	Flatten(),
	Dense(256, activation='relu'),
  Dropout(rate=0.5),  # Dropout oranını artırdık
  Dense(128, activation='relu'),
  Dropout(rate=0.5),  # Dropout oranını artırdık
	Dense(1, activation='sigmoid')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.0001)  # Öğrenme oranını düşürme
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping callback'i oluşturun
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme
history = model.fit(X_train, y_train, batch_size=32, epochs=40, validation_split=0.2, callbacks=[early_stopping])

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/40
16/16 [==============================] - 63s 2s/step - loss: 0.9613 - accuracy: 0.5373 - val_loss: 0.6917 - val_accuracy: 0.5156
Epoch 2/40
16/16 [==============================] - 8s 507ms/step - loss: 0.7132 - accuracy: 0.5255 - val_loss: 0.6924 - val_accuracy: 0.5156
Epoch 3/40
16/16 [==============================] - 8s 520ms/step - loss: 0.6946 - accuracy: 0.5176 - val_loss: 0.6909 - val_accuracy: 0.5938
Epoch 4/40
16/16 [==============================] - 8s 527ms/step - loss: 0.6906 - accuracy: 0.5471 - val_loss: 0.6861 - val_accuracy: 0.6250
Epoch 5/40
16/16 [==============================] - 8s 514ms/step - loss: 0.6900 - accuracy: 0.5294 - val_loss: 0.6779 - val_accuracy: 0.6172
Epoch 6/40
16/16 [==============================] - 8s 515ms/step - loss: 0.6803 - accuracy: 0.5

# Model Save


In [ ]:
from tensorflow.keras.models import load_model
model.save("/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/saved_model/400TrainDataDeneme_VGG16_Blur_Dropout_Early_Model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Cross Validation Eklenmiş Kod DENEME

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping

# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')

# Veri yolu
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Saglikli_Grafikler_Siyah_Beyaz_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Okrali_Grafikler_Siyah_Beyaz_512x512"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Fonksiyon: Görüntü yükleme ve bulanıklık ekleme
def load_and_blur_image(image_path, target_size=(256, 256)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    blurred_img_array = gaussian_filter(img_array, sigma=1.5)  # Bulanıklık ekleme
    return blurred_img_array

# Zararlı sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(okra_klasoru):
    dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme ve bulanıklık ekleme
for dosya_adi in os.listdir(normal_klasoru):
    dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
    img = load_and_blur_image(dosya_yolu)
    img = preprocess_vgg16(img)
    X.append(img)
    y.append(1)  # Normal sınıf

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# K-fold cross validation için k değerini belirleyin
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = False

# Tam bağlı katmanları oluşturma
model = Sequential([
    vgg16_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(rate=0.5),  # Dropout oranını artırdık
    Dense(128, activation='relu'),
    Dropout(rate=0.5),  # Dropout oranını artırdık
    Dense(1, activation='sigmoid')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.0001)  # Öğrenme oranını düşürme
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# K-fold cross validation için modeli eğitme ve değerlendirme
for i, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"Fold {i+1}/{k}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Veri artırma (data augmentation) için ImageDataGenerator oluşturma
    datagen = ImageDataGenerator(
        shear_range=0.1,  # Shirring etkisi
        preprocessing_function=lambda img: gaussian_filter(img, sigma=1.5)  # Gaussian blur ekleme
    )
    # EarlyStopping callback'i oluşturun
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Veri artırma ile modeli eğitme
    history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=80, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy * 100:.2f}%')


IndentationError: unexpected indent (<ipython-input-2-636f616bae83>, line 97)

Deneme1: Loss:0.42 | Acc: %88
Deneme2: Loss:0.40 | Acc: %82
Deneme3: Loss:0.35 | Acc: %83.5

# With GridsearchCV

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD

# Veri yolları
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Stem_Saglikli_Grafikler_Siyah_Beyaz_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Stem_Okrali_Grafikler_Siyah_Beyaz_512x512"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Zararlı sınıf için verileri yükleme
for dosya_adi in os.listdir(okra_klasoru):
    dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
    img = load_img(dosya_yolu, target_size=(256, 256))
    img = img_to_array(img)
    img = preprocess_vgg16(img)  # VGG16 için ön işleme
    X.append(img)
    y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme
for dosya_adi in os.listdir(normal_klasoru):
    dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
    img = load_img(dosya_yolu, target_size=(256, 256))
    img = img_to_array(img)
    img = preprocess_vgg16(img)  # VGG16 için ön işleme
    X.append(img)
    y.append(1)  # Normal sınıf

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = True

# Tam bağlı katmanları oluşturma
model = Sequential([
    vgg16_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Hiperparametre aralıkları tanımlama
param_grid = {
    'optimizer': [Adam(), SGD()],
    'learning_rate': [0.001, 0.0001],
    'batch_size': [16, 32],
    'epochs': [20, 30, 40]
}

# GridSearchCV ile en iyi hiperparametreleri bulma
grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# En iyi parametreler
best_params = grid_search.best_params_
print("En iyi parametreler:", best_params)

# En iyi modeli seçme
model = grid_search.best_estimator_

# Modeli derleme
model.compile(optimizer=best_params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')


# With Early Stopping


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Veri yolları
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Stem_Saglikli_Grafikler_Siyah_Beyaz_512x512"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Stem_Okrali_Grafikler_Siyah_Beyaz_512x512"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Zararlı sınıf için verileri yükleme
for dosya_adi in os.listdir(okra_klasoru):
	dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
	img = load_img(dosya_yolu, target_size=(256, 256))
	img = img_to_array(img)
	#img = preprocess_vgg16(img)  # VGG16 için ön işleme
	X.append(img)
	y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme
for dosya_adi in os.listdir(normal_klasoru):
	dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
	img = load_img(dosya_yolu, target_size=(256, 256))
	img = img_to_array(img)
	#img = preprocess_vgg16(img)  # VGG16 için ön işleme
	X.append(img)
	y.append(1)  # Normal sınıf

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = True

# Tam bağlı katmanları oluşturma
model = Sequential([
	vgg16_model,
	Flatten(),
	Dense(128, activation='relu'),
	Dense(1, activation='sigmoid')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.001)  # Öğrenme oranını düşürme
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping callback'i oluşturun
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')



Epoch 1/40
20/20 [==============================] - 62s 2s/step - loss: 34.7718 - accuracy: 0.5110 - val_loss: 0.6890 - val_accuracy: 0.5938
Epoch 2/40
20/20 [==============================] - 10s 506ms/step - loss: 0.7016 - accuracy: 0.5188 - val_loss: 0.6901 - val_accuracy: 0.5250
Epoch 3/40
20/20 [==============================] - 10s 505ms/step - loss: 0.6878 - accuracy: 0.5596 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 4/40
20/20 [==============================] - 10s 507ms/step - loss: 0.6937 - accuracy: 0.4906 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/40
20/20 [==============================] - 10s 526ms/step - loss: 0.6932 - accuracy: 0.4734 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/40
7/7 [==============================] - 4s 533ms/step - loss: 0.6813 - accuracy: 0.6600
loss: 0.6813051104545593
VGG16 Modeli Test Doğruluk Oranı: 66.00%


# Çoklu sınıflandırma katmanı, Dense 2 Softmax


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam

# Veri yolları
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Deneme_50_PNG/White/Okralı"
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Deneme_50_PNG/White/Normal"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Zararlı sınıf için verileri yükleme
for dosya_adi in os.listdir(okra_klasoru):
	dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
	img = load_img(dosya_yolu)
	img = img_to_array(img)
	img = preprocess_vgg16(img)  # VGG16 için ön işleme
	X.append(img)
	y.append(0)  # Zararlı sınıf

# Normal sınıf için verileri yükleme
for dosya_adi in os.listdir(normal_klasoru):
	dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
	img = load_img(dosya_yolu)
	img = img_to_array(img)
	img = preprocess_vgg16(img)  # VGG16 için ön işleme
	X.append(img)
	y.append(1)  # Normal sınıf

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Verileri karıştırma
X, y = shuffle(X, y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli yükleme
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modeli eğitilemez hale getirme
vgg16_model.trainable = True

# Tam bağlı katmanları oluşturma
model = Sequential([
    vgg16_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Sigmoid yerine softmax kullanıyoruz
])

# Modeli derleme
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Etiketleri ikili biçimden kategorik biçime dönüştürme
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Modeli eğitme
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'VGG16 Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')


Epoch 1/20
3/3 [==============================] - 5s 481ms/step - loss: 147.6290 - accuracy: 0.5139 - val_loss: 548.4276 - val_accuracy: 0.3750
Epoch 2/20
3/3 [==============================] - 1s 373ms/step - loss: 207.9905 - accuracy: 0.4861 - val_loss: 2.5562 - val_accuracy: 0.6250
Epoch 3/20
3/3 [==============================] - 1s 386ms/step - loss: 5.9248 - accuracy: 0.5694 - val_loss: 0.9793 - val_accuracy: 0.6250
Epoch 4/20
3/3 [==============================] - 1s 371ms/step - loss: 1.6571 - accuracy: 0.4583 - val_loss: 0.7184 - val_accuracy: 0.3750
Epoch 5/20
3/3 [==============================] - 1s 368ms/step - loss: 0.6942 - accuracy: 0.5694 - val_loss: 1.6137 - val_accuracy: 0.6250
Epoch 6/20
3/3 [==============================] - 1s 361ms/step - loss: 1.4380 - accuracy: 0.5139 - val_loss: 0.6818 - val_accuracy: 0.6250
Epoch 7/20
3/3 [==============================] - 1s 361ms/step - loss: 0.7446 - accuracy: 0.5139 - val_loss: 0.6560 - val_accuracy: 0.6250
Epoch 8/20
3/3